In [1]:
import pandas as pd

In [2]:
# df = pd.read_csv("https://info.dengue.mat.br/datasets/notificacao/20102023/weather_notification_SC_2010-01-01_2023-08-30.csv")

In [3]:
url = "/workspace/repos/cursos/ml-dengue-predict/fetchinfodenguedata/dataset/bkp/weather_notification_SC_2010-01-01_2023-08-30.csv"

df = pd.read_csv(url,index_col=[0]).reset_index(drop=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522809 entries, 0 to 522808
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   uf              522809 non-null  object 
 1   geocodigo       522809 non-null  int64  
 2   nome_municipio  522809 non-null  object 
 3   dt_notific      522809 non-null  object 
 4   se_notif        522809 non-null  int64  
 5   ano_notif       522809 non-null  int64  
 6   temp_med        522809 non-null  float64
 7   precip_med      522809 non-null  float64
 8   pressao_med     522809 non-null  float64
 9   umid_med        522809 non-null  float64
dtypes: float64(4), int64(3), object(3)
memory usage: 39.9+ MB


In [4]:
df.head()

,uf,geocodigo,nome_municipio,dt_notific,se_notif,ano_notif,temp_med,precip_med,pressao_med,umid_med
0,Santa Catarina,4209102,Joinville,2023-04-30,18,2023,20.466202,0.035414,1.005391,87.35066
1,Santa Catarina,4209102,Joinville,2023-04-30,18,2023,20.466202,0.035414,1.005391,87.35066
2,Santa Catarina,4209102,Joinville,2023-04-30,18,2023,20.466202,0.035414,1.005391,87.35066
3,Santa Catarina,4209102,Joinville,2023-04-30,18,2023,20.466202,0.035414,1.005391,87.35066
4,Santa Catarina,4209102,Joinville,2023-04-30,18,2023,20.466202,0.035414,1.005391,87.35066


In [5]:
list(df.columns)

['uf',
 'geocodigo',
 'nome_municipio',
 'dt_notific',
 'se_notif',
 'ano_notif',
 'temp_med',
 'precip_med',
 'pressao_med',
 'umid_med']

In [9]:
import pandas as pd

cols = ['uf', 'geocodigo', 'nome_municipio', 'dt_notific', 'se_notif', 'ano_notif', 'temp_med', 'umid_med']


df_combined = df[cols] \
    .groupby(['uf', 'nome_municipio', 'se_notif', 'ano_notif', 'geocodigo']) \
    .agg(casos_notif=('dt_notific', 'count'),
         temp_med=('temp_med', 'mean'),
         umid_med=('umid_med', 'mean')) \
    .reset_index() \
    .sort_values(['ano_notif', 'se_notif'], ascending=[False, False])

df_combined


,uf,nome_municipio,se_notif,ano_notif,geocodigo,casos_notif,temp_med,umid_med
334,Santa Catarina,Apiúna,33,2023,4201257,1,19.904106,82.034000
483,Santa Catarina,Araquari,33,2023,4201307,10,19.094256,87.402260
852,Santa Catarina,Balneário Barra do Sul,33,2023,4202057,4,18.981406,91.471262
1235,Santa Catarina,Balneário Camboriú,33,2023,4202008,10,16.144434,80.873799
1583,Santa Catarina,Balneário Piçarras,33,2023,4212809,3,14.223744,79.497622
...,...,...,...,...,...,...,...,...
6390,Santa Catarina,Florianópolis,1,2010,4205407,2,25.724470,84.835140
9255,Santa Catarina,Itajaí,1,2010,4208203,1,25.072872,82.722810
11177,Santa Catarina,Joinville,1,2010,4209102,2,25.964615,85.863235
11787,Santa Catarina,Lages,1,2010,4209300,1,20.545506,93.905365


In [10]:
X = df_combined[['temp_med', 'umid_med', 'se_notif', 'ano_notif', 'geocodigo']]
y = df_combined['casos_notif']


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
from sklearn.ensemble import RandomForestRegressor

# Crie uma instância do modelo com os hiperparâmetros desejados
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Treine o modelo com os dados de treinamento
rf_model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [14]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = rf_model.predict(X_test)

# Calcule o RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)

# Calcule o R-squared
r_squared = r2_score(y_test, y_pred)

print(f'RMSE: {rmse}')
print(f'R-squared: {r_squared}')


RMSE: 65.2691993262952
R-squared: 0.9012074286370864
